## LDA code setup

In [2]:

import nltk as nltk
import openpyxl as openpyxl
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import *
from sklearn.metrics.pairwise import cosine_similarity
from openpyxl import Workbook
import nltk
import pandas as pd
from numpy import array
from collections import Counter
#from gensim.models import Word2Vec

from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import s_
import datetime


In [94]:
reviews = pd.read_csv('reviews_till_Date.csv')

In [95]:
reviews.head()

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,0,gp:AOqpTOEyU7-40Ras_s2PEETG3BRpqcnX70XPSMadRe3...,Adebola Babatunde,https://play-lh.googleusercontent.com/a/AATXAJ...,there should be an option for a chat,3,0,1.0.11,2021-12-02 19:01:45,NaN,NaN
1,1,gp:AOqpTOE8mZw_WPC2DJbWApajZNvT1rwy3EacEG87UcR...,James maliet manyuat,https://play-lh.googleusercontent.com/a/AATXAJ...,very fantastic app,5,0,1.0.18,2021-12-02 18:36:37,NaN,NaN
2,2,gp:AOqpTOE7ocCUUOQQWaUBYc7UPcSDwBbyGF35bdXeUSa...,Tofighe Mokhtari,https://play-lh.googleusercontent.com/a-/AOh14...,Good,5,0,1.0.16,2021-12-02 18:05:22,NaN,NaN
3,3,gp:AOqpTOE4axGizzCOXlL9cKplteffne0vG_wtREXlmqZ...,mehran aslani,https://play-lh.googleusercontent.com/a-/AOh14...,🤦,2,0,1.0.16,2021-12-02 17:43:50,NaN,NaN
4,4,gp:AOqpTOFda4wLwvfII1Vo1tvZ3kic07FivwVuS4mPmax...,Thelma Smiles,https://play-lh.googleusercontent.com/a-/AOh14...,cool but always having network problems,4,0,1.0.18,2021-12-02 17:31:46,NaN,NaN


In [96]:
#changing the name of 1st column and the date column
reviews.rename(columns={reviews.columns[0]: "s_no", "at": "date_posted"}, inplace = True)


In [97]:
reviews.head()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
0,0,gp:AOqpTOEyU7-40Ras_s2PEETG3BRpqcnX70XPSMadRe3...,Adebola Babatunde,https://play-lh.googleusercontent.com/a/AATXAJ...,there should be an option for a chat,3,0,1.0.11,2021-12-02 19:01:45,NaN,NaN
1,1,gp:AOqpTOE8mZw_WPC2DJbWApajZNvT1rwy3EacEG87UcR...,James maliet manyuat,https://play-lh.googleusercontent.com/a/AATXAJ...,very fantastic app,5,0,1.0.18,2021-12-02 18:36:37,NaN,NaN
2,2,gp:AOqpTOE7ocCUUOQQWaUBYc7UPcSDwBbyGF35bdXeUSa...,Tofighe Mokhtari,https://play-lh.googleusercontent.com/a-/AOh14...,Good,5,0,1.0.16,2021-12-02 18:05:22,NaN,NaN
3,3,gp:AOqpTOE4axGizzCOXlL9cKplteffne0vG_wtREXlmqZ...,mehran aslani,https://play-lh.googleusercontent.com/a-/AOh14...,🤦,2,0,1.0.16,2021-12-02 17:43:50,NaN,NaN
4,4,gp:AOqpTOFda4wLwvfII1Vo1tvZ3kic07FivwVuS4mPmax...,Thelma Smiles,https://play-lh.googleusercontent.com/a-/AOh14...,cool but always having network problems,4,0,1.0.18,2021-12-02 17:31:46,NaN,NaN


In [111]:
#adjusting the date column to datetime format for filtering purpose in future
reviews["date_posted"] = pd.to_datetime(reviews["date_posted"], format='%Y-%m-%d')

In [112]:
reviews["date_posted"].head()

0   2021-12-02
1   2021-12-02
2   2021-12-02
3   2021-12-02
4   2021-12-02
Name: date_posted, dtype: datetime64[ns]

In [108]:
#testing the date filters
reviews.loc[reviews['date_posted'] >= "2021-12-02"]

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
0,0,gp:AOqpTOEyU7-40Ras_s2PEETG3BRpqcnX70XPSMadRe3...,Adebola Babatunde,https://play-lh.googleusercontent.com/a/AATXAJ...,there should be an option for a chat,3,0,1.0.11,2021-12-02 19:01:45,NaN,NaN
1,1,gp:AOqpTOE8mZw_WPC2DJbWApajZNvT1rwy3EacEG87UcR...,James maliet manyuat,https://play-lh.googleusercontent.com/a/AATXAJ...,very fantastic app,5,0,1.0.18,2021-12-02 18:36:37,NaN,NaN
2,2,gp:AOqpTOE7ocCUUOQQWaUBYc7UPcSDwBbyGF35bdXeUSa...,Tofighe Mokhtari,https://play-lh.googleusercontent.com/a-/AOh14...,Good,5,0,1.0.16,2021-12-02 18:05:22,NaN,NaN
3,3,gp:AOqpTOE4axGizzCOXlL9cKplteffne0vG_wtREXlmqZ...,mehran aslani,https://play-lh.googleusercontent.com/a-/AOh14...,🤦,2,0,1.0.16,2021-12-02 17:43:50,NaN,NaN
4,4,gp:AOqpTOFda4wLwvfII1Vo1tvZ3kic07FivwVuS4mPmax...,Thelma Smiles,https://play-lh.googleusercontent.com/a-/AOh14...,cool but always having network problems,4,0,1.0.18,2021-12-02 17:31:46,NaN,NaN
5,5,gp:AOqpTOEq5oxzZAJj1D4ByLrUykIzd3Q1V3UE6Y00LZV...,Hussain Alhussain,https://play-lh.googleusercontent.com/a/AATXAJ...,𝗇𝗂𝖼𝖾 𝖺𝗉𝗉,5,0,1.0.18,2021-12-02 16:08:57,NaN,NaN
6,6,gp:AOqpTOH_UNmZbGx_WV-OGgDeM8mp4NhT_NAH_A2mbGE...,Mustapha Hadjoudja,https://play-lh.googleusercontent.com/a/AATXAJ...,Amazing,5,0,NaN,2021-12-02 15:24:01,NaN,NaN
7,7,gp:AOqpTOFBhPdyhUCw6YDVDWKvOrV0TPLfbUF4_f1s6G8...,guard Justice5,https://play-lh.googleusercontent.com/a/AATXAJ...,This app store your conversation and dont allo...,1,0,NaN,2021-12-02 15:07:08,NaN,NaN
8,8,gp:AOqpTOFWEq0aDq3v1dZVncoVMYkrne31rl-i1UTpUdZ...,David Dominic,https://play-lh.googleusercontent.com/a/AATXAJ...,harry fabrice club house is perfect,5,0,1.0.18,2021-12-02 14:57:30,NaN,NaN
9,9,gp:AOqpTOHRTaIGLof87XFpxALkCj2Nx9JafHwLjrUe9fw...,Mashallah Shamsolvaezin,https://play-lh.googleusercontent.com/a/AATXAJ...,کلاب هاوس به اینترنت قوی نیاز دارد درحالیکه وا...,2,0,1.0.16,2021-12-02 14:38:00,NaN,NaN


In [114]:

lemmatizer = nltk.stem.WordNetLemmatizer()

#Step 1: Transform reviews into term doc matrix, lemmatize, remove stop words etc
processed_reviews = []
original_reviews = []
for each_review in reviews["content"]:
    original_reviews.append(each_review)
    review_token = nltk.word_tokenize(each_review.lower())
    review_token = [lemmatizer.lemmatize(token) for token in review_token if token.isalpha()]
    review_token = [token for token in review_token if not token in stopwords.words('english')]
    processed_reviews.append(review_token)


['good']


In [119]:
print(processed_reviews[3])

[]


In [118]:
original_reviews[0:4]

['there should be an option for a chat', 'very fantastic app', 'Good', '🤦']

In [120]:
#tf-idf
Tfidf_input = []
for i in processed_reviews:
    Tfidf_input.append(" ".join(i))

#applying unigram and bigram and keeping term doc frequency = 4
review_vect = TfidfVectorizer(ngram_range=(1,2), min_df=4)
review_vect.fit(Tfidf_input)
review_vect2 = review_vect.transform(Tfidf_input)


In [121]:
print(review_vect2.toarray())
print(review_vect2.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(29325, 6371)


In [124]:
#Step 2: Use LDA model to extract topics of each document
terms = review_vect.get_feature_names()
lda = LatentDirichletAllocation(n_components=10).fit(review_vect2)

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    topic_dict[topic_idx] = ",".join([terms[i] for i in topic.argsort()[-10:]])
    print("Topic %d:" % (topic_idx))
    print(",".join([terms[i] for i in topic.argsort()[:-10-1:-1]]))

Topic 0:
great,best,superb,app,application,clubhouse,number,fun,use,wa
Topic 1:
super,waste,platform,app,time,ever,app ever,better,good platform,waste time
Topic 2:
open,gud,club,app,house,poor,club house,one,idea,beautiful
Topic 3:
good,nice,awesome,nice app,amazing,love,app,working,awesome app,amazing app
Topic 4:
ok,excellent,perfect,change,name,عالی,work,option,change name,room
Topic 5:
app,bad,good app,good,cool,loved,experience,bad app,delete,good experience
Topic 6:
worst,useful,app,social,medium,worst app,one,social medium,good one,voice
Topic 7:
app,great app,great,add,fantastic,wonderful,instagram,please,good,option
Topic 8:
like,app,best app,best,dark,mode,error,dark mode,hear,problem
Topic 9:
account,get,interesting,wow,bug,still,code,invite,ca,app


In [122]:
terms = review_vect.get_feature_names()
terms

['aa',
 'aap',
 'aap hai',
 'aapp',
 'aaya',
 'ab',
 'abhi',
 'ability',
 'ability change',
 'able',
 'able access',
 'able add',
 'able change',
 'able connect',
 'able create',
 'able delete',
 'able download',
 'able enter',
 'able follow',
 'able get',
 'able hear',
 'able invite',
 'able join',
 'able link',
 'able listen',
 'able log',
 'able login',
 'able make',
 'able open',
 'able participate',
 'able register',
 'able see',
 'able sign',
 'able talk',
 'able update',
 'able use',
 'abruptly',
 'absolute',
 'absolutely',
 'absolutely awesome',
 'absolutely love',
 'absurd',
 'abt',
 'abuse',
 'abusing',
 'abusive',
 'abusive language',
 'ac',
 'acc',
 'acc ready',
 'accept',
 'acceptable',
 'accepted',
 'accepting',
 'access',
 'access account',
 'access app',
 'access contact',
 'accessibility',
 'accessible',
 'accessing',
 'accidentally',
 'accont',
 'according',
 'according interest',
 'accordingly',
 'account',
 'account able',
 'account account',
 'account activated',
 